In [1]:
# As first_reg, but with features and labels

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass
    
    
class NearestInterp(FineGridPredictorMixin, NearestInterpBase):
    pass


class PolyInterp(FineGridPredictorMixin, PolyInterpBase):
    pass
    
    
class ScaleData(Processor):
    """Scales data to between -1 and 1."""
    
    def __init__(self, *args, **kwargs):
        self.X_mean = None
        self.X_extent = None
        super(ScaleData, self).__init__(*args, **kwargs)
        
    def transform(self, X, y):
        X_scaled, y_scaled, X_mean, X_extent = self._transform(X, y)
        self.X_mean = X_mean
        self.X_extent = X_extent
        
        return X_scaled, y_scaled
    
    @staticmethod
    def _transform(X, y):
        X_mean = np.mean(X)
        X_from_mean = X - X_mean
        X_extent = np.max(np.abs(X_from_mean))
        X_scaled = X_from_mean / X_extent
        y_scaled = (y - X_mean) / X_extent
        return X_scaled, y_scaled, X_mean, X_extent
    
    @classmethod
    def once(cls, X, y):
        """One-off call to perform a transform without the overheard
        of creating the class.
        """
        X_scaled, y_scaled, X_mean, X_extent = cls._transform(X, y)
        return X_scaled, y_scaled
    
    def _inverse_transform(self, y):
        return (y * self.X_extent) + self.X_mean
        
    
    
def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the feature is the solution on a coarse grid, and the
    label is the solution on a fine grid, with both of them scaled.
    """
    
    X, y = gen_one_peakon()
    return ScaleData.once(X, y)

In [5]:
### DNN hyperparameters
hidden_units = [1000] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir = './saved_models/seventh_reg/'
model_dir = model_dir_str(model_dir, hidden_units, logits, drop_rate)

dnn = Sequential.create_dnn(hidden_units, logits, drop_rate=drop_rate,
                            model_dir=model_dir, log_steps=log_steps,
                            gradient_clip=gradient_clip)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/seventh_reg/1000x10_121_D04', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd865702a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 50
    steps = 2000
    train_input_fn = IdentityProcess().init(BatchData(gen_one_data, batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps)
    
#     batch_size = 100
#     steps = 20000
#     train_input_fn = BatchData(gen_one_data, batch_size)
#     dnn.train(input_fn=train_input_fn, max_steps=steps)
    
#     batch_size = 1000
#     steps = 25000
#     train_input_fn = BatchData(gen_one_data, batch_size)
#     dnn.train(input_fn=train_input_fn, max_steps=steps)
    
#     batch_size = 5000
#     steps = 35000
#     train_input_fn = BatchData(gen_one_data, batch_size)
#     dnn.train(input_fn=train_input_fn, max_steps=steps)

INFO:tensorflow:Skipping training since max_steps has already saved.


In [22]:
X, y = gen_one_peakon()

In [23]:
poly_deg = 5
results = plot_regressors([(Perfect(), None, 'Exact'),
                           (dnn, ScaleData(), 'DNN'),
                           (BilinearInterp(), None, 'Bilinear'),
                           (PolyInterp(poly_deg), None, 'Poly-{}'.format(poly_deg))],
                          X,
                          y)

FigureCanvasNbAgg()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/seventh_reg/1000x10_121_D04/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
result[0].loss, result[1].loss, result[2].loss

(0.13680501680071605, 2.68223820688442, 84.18740987394793)